In [101]:
import numpy as np
import pandas as pd
import re
import math
from collections import Counter
import nltk
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/MCM/Problem_C_Data_Wordle_updated.xlsx")

In [99]:
df['Number of  reported results']

0       20380
1       21204
2       20001
3       20160
4       20879
        ...  
354    153880
355    107134
356     91477
357    101503
358     80630
Name: Number of  reported results, Length: 359, dtype: int64

In [65]:
df2 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/MCM/regression_version.xlsx")
df2.iloc[5,]

Date                           2022-12-26 00:00:00
Contest number                                 555
Word                                         judge
Number of  reported results                  20011
Number in hard mode                           2043
1 try                                            0
2 tries                                          2
3 tries                                          8
4 tries                                         16
5 tries                                         26
6 tries                                         33
7 or more tries (X)                             14
Sum                                             99
Average Score                             5.232323
Length                                           5
Hard Mode Percent                         0.102094
Name: 5, dtype: object

In [ ]:
# word_list = []
# word_list = df['Word']
# alpha_pos = [[0,0,0,0,0] for i in range(26)]
# alpha_list = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
# #traverse the list
# for i in range(len(word_list)):
#   #traverse the strings
#   for j in range(len(word_list[i])):
#     for k in range(len(alpha_list)):
#       if word_list[i][j] == alpha_list[k]:
#         alpha_pos[k][j] += 1

# col_1 = [i[0] for i in alpha_pos]
# col_2 = [i[1] for i in alpha_pos]
# col_3 = [i[2] for i in alpha_pos]
# col_4 = [i[3] for i in alpha_pos]
# col_5 = [i[4] for i in alpha_pos]
# df2 = pd.DataFrame(alpha_pos)

# sum_freq = []
# for i in alpha_pos:
#   sum_freq.append(sum(i))
# sum_col = pd.DataFrame(sum_freq)

# df_lst = [pd.DataFrame(alpha_list), df2, sum_col]
# charpos = pd.concat(df_lst, axis = 1)

In [78]:
file = open("/content/drive/MyDrive/Colab Notebooks/MCM/untitled.txt")
#words = file.read().split()
words = df.Word
CZERO = ord('a')

In [5]:
charpos=np.zeros((26,6))
#traverse the list
for i in range(len(words)):
    #traverse the strings
    for j in range(5):
       charpos[ord(words[i][j])-CZERO][j]+=1
       charpos[ord(words[i][j])-CZERO][5]+=1

In [6]:
charnumber = np.zeros(len(words))
count = 0
for i in range(len(words)):
    count = 0
    word = words[i]
    for j in range(5):
        same = False
        for k in range(j):
            if word[k] == word[j]:
                same = True
                break
        if not same:
            count += 1
    charnumber[i] = count

In [7]:
charcombo = np.zeros((26, 26))
for i in range(len(words)):
    for j in range(4):
        a = ord(words[i][j]) - ord("a")
        b = ord(words[i][j + 1]) - ord("a")
        charcombo[a][b] += 1

In [8]:
vowel_list = ['a','e','i','o','u']
total_vowel = 0
for i in words:
  for j in i:
    if j in vowel_list:
      total_vowel += 1
total_vowel

4097

In [ ]:
# charpos.columns = ["alpha", "pos1", "pos2", "pos3", "pos4", "pos5", "sum"]
# charpos

In [9]:
charpos

array([[1.410e+02, 3.030e+02, 3.060e+02, 1.620e+02, 6.300e+01, 9.750e+02],
       [1.730e+02, 1.600e+01, 5.600e+01, 2.400e+01, 1.100e+01, 2.800e+02],
       [1.980e+02, 4.000e+01, 5.600e+01, 1.500e+02, 3.100e+01, 4.750e+02],
       [1.110e+02, 2.100e+01, 7.500e+01, 7.000e+01, 1.180e+02, 3.950e+02],
       [7.200e+01, 2.420e+02, 1.770e+02, 3.190e+02, 4.210e+02, 1.231e+03],
       [1.340e+02, 8.000e+00, 2.500e+01, 3.500e+01, 2.600e+01, 2.280e+02],
       [1.150e+02, 1.100e+01, 6.700e+01, 7.600e+01, 4.100e+01, 3.100e+02],
       [6.900e+01, 1.440e+02, 9.000e+00, 2.800e+01, 1.370e+02, 3.870e+02],
       [3.400e+01, 2.010e+02, 2.670e+02, 1.580e+02, 1.100e+01, 6.710e+02],
       [2.000e+01, 2.000e+00, 3.000e+00, 2.000e+00, 0.000e+00, 2.700e+01],
       [2.000e+01, 1.000e+01, 1.200e+01, 5.500e+01, 1.130e+02, 2.100e+02],
       [8.700e+01, 2.000e+02, 1.110e+02, 1.620e+02, 1.550e+02, 7.150e+02],
       [1.080e+02, 3.800e+01, 6.100e+01, 6.800e+01, 4.200e+01, 3.170e+02],
       [3.700e+01, 8.700e

In [10]:
charnumber

array([4., 3., 4., ..., 5., 5., 5.])

In [11]:
charcombo

array([[  0.,  29.,  52.,  50.,   1.,  18.,  25.,   2.,  47.,   2.,  30.,
        114.,  43.,  93.,   3.,  35.,   0., 107.,  71.,  68.,  27.,  30.,
         22.,   7.,  23.,  13.],
       [ 35.,   8.,   0.,   0.,  44.,   0.,   0.,   1.,  25.,   0.,   0.,
         40.,   0.,   0.,  39.,   0.,   0.,  39.,   0.,   1.,  27.,   0.,
          0.,   0.,  10.,   0.],
       [ 54.,   0.,   2.,   0.,  44.,   0.,   0., 103.,  11.,   0.,  54.,
         34.,   0.,   0.,  46.,   0.,   0.,  55.,   0.,  13.,  20.,   0.,
          0.,   0.,   8.,   0.],
       [ 20.,   0.,   0.,  13.,  78.,   0.,  14.,   0.,  30.,   0.,   1.,
          7.,   2.,   0.,  25.,   0.,   0.,  30.,   2.,   1.,  11.,   0.,
          3.,   0.,  40.,   0.],
       [ 89.,  11.,  24.,  40.,  59.,  13.,  15.,   1.,  14.,   1.,   5.,
         84.,  22.,  77.,   3.,  24.,   2., 189.,  26.,  53.,   4.,  16.,
         11.,  12.,  14.,   1.],
       [ 14.,   0.,   0.,   0.,  26.,  20.,   0.,   0.,  27.,   1.,   0.,
         37.,   0.,  

In [ ]:
# def getfeature(word):
#     # charnumber
#     f1 = 0
#     for j in range(5):
#         same = False
#         for k in range(j):
#             if word[k] == word[j]:
#                 same = True
#                 break
#         if not same:
#             f1 += 1

#     # charfeq
#     f2 = 0
#     for i in range(5):
#         f2 += charpos[ord(word[i])][6]

#     # charpos
#     f3 = 0
#     for i in range(5):
#         f3 += charpos[ord(word[i])][i]

#     # charcombo
#     f4 = 0
#     for i in range(4):
#         a = ord(word[i]) - ord("a")
#         b = ord(word[i + 1]) - ord("a")
#         f4+=charcombo[a][b]

#     return [f1,f2,f3,f4]

In [41]:
def getfeature(word):
    # charnumber
    f1 = 0
    for j in range(5):
        same = False
        for k in range(j):
            if word[k] == word[j]:
                same = True
                break
        if not same:
            f1 += 1
    charnumber_counter = Counter(charnumber)
    f1 = charnumber_counter[f1] / len(charnumber)

    # charfeq
    f2 = 0
    for i in range(5):
        f2 += charpos[ord(word[i]) - ord("a")][5]
    f2 /= np.sum(charpos[:, -1])

    # charpos
    f3 = 0
    for i in range(5):
        f3 += charpos[ord(word[i]) - ord("a")][i]
    f3 /= (np.sum(charpos[:, 0]) * 5)

    # charcombo
    f4 = 0
    for i in range(4):
        a = ord(word[i]) - ord("a")
        b = ord(word[i + 1]) - ord("a")
        f4 += charcombo[a][b]
    f4 /= np.sum(charcombo)

    return [f1,f2,f3,f4]

In [42]:
featureList = []
for word in words:
    featureList.append(getfeature(word))
len(featureList)

359

In [13]:
f1_coef = 0
for k, v in Counter(charnumber).items():
    p = v / np.sum(charnumber)
    f1_coef -= p * np.log2(p)
f1_coef

0.7000715075677838

In [14]:
f2_coef = 0
freq = charpos[:, -1] / np.sum(charpos[:, -1])
for p in freq:
    f2_coef -= p * np.log2(p)
f2_coef

4.3273895621070295

In [15]:
f3_coef = 0
freq = charpos[:, :-1] / np.sum(charpos[:, 0])
freq = np.sum(freq, axis=1)
for p in freq:
    f3_coef -= p * np.log2(p)
f3_coef

10.027307336098337

In [16]:
f4_coef = 0
freq = charcombo / np.sum(charcombo)
freq = freq.reshape(-1, 1)
for p in freq:
    if p > 0.:
        f4_coef -= p * np.log2(p)
f4_coef = f4_coef[0]
f4_coef

7.828576791300392

In [17]:
def calculate_score(w):
    f = getfeature(w)
    return f[0] * f1_coef + f[1] * f2_coef + f[2] * f3_coef + f[3] * f4_coef

In [54]:
words_score = {}
for word in words:
    score = calculate_score(word)
    words_score[word] = score
    #print(f"{word}: {score}")

In [30]:
scoreList = []
for i in words_score:
    scoreList.append(words_score[i])
len(scoreList)

359

In [55]:
words_score_sorted = sorted(words_score.items(), key=lambda x: x[1], reverse=True)
#words_score_sorted

In [51]:
from sklearn import svm
X = featureList
y = df2["Average Score"]
regr = svm.SVR()
regr.fit(X, y)

SVR()

In [59]:
#regr.predict(np.array(getfeature('eerie')))
input = np.array(getfeature('condo')).reshape(1,-1)
regr.predict(input)[0]

4.478035657804078

In [58]:
regr.score(X,y)

0.2474931940252033

In [61]:
predicted_value = []
for word in words:
    predicted_value.append(regr.predict(np.array(getfeature(word)).reshape(1,-1))[0])

In [67]:
diff_lst = predicted_value - y
#for i in diff_lst:
  #print(i)
np.corrcoef(predicted_value,scoreList)

array([[ 1.        , -0.58244225],
       [-0.58244225,  1.        ]])

In [76]:
#分析词性
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
words = nltk.word_tokenize(file.read())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [81]:
print(nltk.pos_tag(words))

[('manly', 'RB'), ('molar', 'JJ'), ('havoc', 'NN'), ('impel', 'NN'), ('condo', 'NN'), ('judge', 'NN'), ('extra', 'JJ'), ('poise', 'NN'), ('aorta', 'NN'), ('excel', 'NN'), ('lunar', 'JJ'), ('third', 'JJ'), ('slate', 'NN'), ('taper', 'IN'), ('chord', 'NN'), ('probe', 'NN'), ('rival', 'JJ'), ('usual', 'JJ'), ('spoke', 'VBD'), ('apply', 'RB'), ('naive', 'JJ'), ('knock', 'NN'), ('braid', 'NN'), ('infer', 'NN'), ('joust', 'NN'), ('amber', 'VB'), ('woken', 'VBN'), ('adore', 'RB'), ('torso', 'JJ'), ('chafe', 'NN'), ('eject', 'NN'), ('study', 'NN'), ('undue', 'JJ'), ('tepid', 'NNS'), ('happy', 'JJ'), ('clean', 'JJ'), ('itchy', 'NN'), ('feast', 'NN'), ('drive', 'NN'), ('prime', 'JJ'), ('axiom', 'NN'), ('brave', 'VBP'), ('avert', 'JJ'), ('glyph', 'NN'), ('there', 'EX'), ('baker', 'NN'), ('snarl', 'NN'), ('maple', 'NN'), ('inane', 'NN'), ('valet', 'NN'), ('medal', 'NNP'), ('unite', 'JJ'), ('rainy', 'NN'), ('spell', 'NN'), ('begin', 'VB'), ('stale', 'JJ'), ('dream', 'NN'), ('photo', 'NN'), ('aloud'

In [83]:
from pandas.core.indexes import category
category = []
for i in nltk.pos_tag(words):
    category.append(i[1])

In [85]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

syllable_lst = []
for i in words:
    syllable_lst.append(syllable_count(i))
len(syllable_lst)


359

In [123]:
# load the time series data into a pandas DataFrame
#data = pd.read_csv('time_series_data.csv', index_col=0, parse_dates=True)
# define the function to perform the ADF test
def adf_test(timeseries):
    """
    Perform Augmented Dickey-Fuller test on a time series
    """
    # calculate rolling statistics
    rolmean = timeseries.rolling(window=12).mean()
    rolstd = timeseries.rolling(window=12).std()

    # perform the ADF test
    adf_result = adfuller(timeseries, autolag='AIC')
    print(f'ADF Statistic: {adf_result[0]}')
    print(f'p-value: {adf_result[1]}')
    print('Critical Values:')
    for key, value in adf_result[4].items():
        print(f'{key}: {value}')

    # plot the rolling statistics
    plt.plot(timeseries, color='blue', label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()

# call the function to perform the ADF test on the time series data
#df["Number of  reported results"]
#rev_col = df["Number of  reported results"].values.tolist().reverse()
rev_col = df["Number of  reported results"].iloc[::-1].reset_index()
rev_col = rev_col.drop(columns=['index'])
rev_col
#print(rev_col)
#adf_test(rev_col)

,Number of reported results
0,80630
1,101503
2,91477
3,107134
4,153880
...,...
354,20879
355,20160
356,20001
357,21204


In [158]:
data

In [ ]:
data = np.log(rev_col)
# fit an ARIMA model to the time series data
p_list = range(1, 6)
q_list = range(1, 6)
d_list = range(2)

for p in p_list:
  for q in q_list:
    for d in d_list:
      print(f"{p} {d} {q}")

      model = ARIMA(data, order=(1,1,1))
      results = model.fit()

      # print a summary of the model results
      print(results.summary())

      # plot the residuals of the model
      residuals = pd.DataFrame(results.resid)
      residuals.plot()
      plt.show()

      # plot the actual values of the time series data compared to the predicted values from the model
      fig, ax = plt.subplots(figsize=(6,3))
      ax.plot(data, label='Actual')
      ax.plot(results.predict(start=data.index[0], end=data.index[-1]), label='Predicted')
      ax.legend(loc='upper left')
      plt.show()

In [156]:
lst = []

#forecast, std_err, conf_int = results.get_forecast(steps=1, alpha=0.05)
obj = results.get_forecast(steps=1, alpha=0.05)
obj.se_mean
#print(lst)
#print(f'Forecast: {forecast[0]}')
#print(f'Standard error: {std_err[0]}')
#print(f'95% Prediction Interval: {conf_int[0]} - {conf_int[1]}')

359    11710.148084
dtype: float64